In [1]:
import os

In [2]:
from ape import Contract, accounts, chain, networks

C:\Users\chuck.raghavan\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dotenv import find_dotenv, load_dotenv

In [4]:
from addresses import ADDRESSES

In [5]:
load_dotenv(find_dotenv())

True

In [6]:
dotenv_path = find_dotenv()
print(f"Dotenv path: {dotenv_path}")

Dotenv path: C:\Users\chuck.raghavan\OneDrive - Los Angeles Unified School District\Documents\GitHub\EncodeBootcamps\Gizathon\.env


In [7]:
dev_passphrase = os.environ.get("DEV_PASSPHRASE")
sepolia_rpc_url = os.environ.get("SEPOLIA_RPC_URL")

In [8]:
networks.parse_network_choice(f"ethereum:sepolia:{sepolia_rpc_url}").__enter__()

INFO: Connecting to existing Geth node at https://eth-sepolia.g.alchemy.com/[hidden].


<geth chain_id=11155111>

In [9]:
chain_id = chain.chain_id
chain_id

11155111

In [10]:
ADDRESSES["WBTC"]

{1: '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
 11155111: '0x779877A7B0D9E8603169DdbD7836e478b4624789',
 5: '0xB4FBF271143F4FBf7B91A5ded31805e42b2208d6',
 42161: '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'}

In [11]:
wbtcAddr = ADDRESSES["WBTC"][chain_id]
wbtcAddr

'0x779877A7B0D9E8603169DdbD7836e478b4624789'

In [ ]:
# Contract('0x473E58A0D2BcD5d985ab083dC6883f66c24EEFB6')

In [13]:
weth_mint_amount = 0.01
pool_fee = 3000
uni = Contract(ADDRESSES["UNI"][chain_id])
weth = Contract(ADDRESSES["WETH"][chain_id])
# wbtc = Contract(ADDRESSES["WETH"][chain_id])
wusdc = Contract(ADDRESSES["USDC"][chain_id])
# wbtc = Contract('0x66194f6c999b28965e0303a84cb8b797273b6b8b')
weth_decimals = weth.decimals()
# wbtc_decimals = wbtc.decimals()
uni_decimals = uni.decimals()
wusdc_decimals = wusdc.decimals()
weth_mint_amount = int(weth_mint_amount * 10**weth_decimals)
uni_mint_amount = int(0.5 * weth_mint_amount)
# wbtc_mint_amount = int(0.5 * weth_mint_amount)

In [14]:
pool_factory = Contract(ADDRESSES["PoolFactory"][chain_id])
pool_address = "0x287B0e934ed0439E2a7b1d5F0FC25eA2c24b64f7"
pool = Contract(pool_address)
swap_router = Contract(ADDRESSES["Router"][chain_id])
wallet = accounts.load("giza1")
wallet.set_autosign(True, passphrase=dev_passphrase)

In [15]:
wallet.address

'0x39BaE1e8dEc32d94181bCDa1DC25218Ef57a12c2'

In [ ]:
with accounts.use_sender("giza1"):
    print(f"Minting {weth_mint_amount/10**weth_decimals} WETH")
    weth.deposit(value=weth_mint_amount)
    print("Approving WETH for swap")
    weth.approve(swap_router.address, weth_mint_amount)
    swap_params = {
        "tokenIn": weth.address,
        "tokenOut": uni.address,
        "fee": pool_fee,
        "recipient": wallet.address,
        "amountIn": weth_mint_amount,
        "amountOutMinimum": 0,
        "sqrtPriceLimitX96": 0,
    }
    swap_params = tuple(swap_params.values())
    print("Swapping WETH for UNI")
    amountOut = swap_router.exactInputSingle(swap_params)
    print(f"Successfully minted {uni_mint_amount/10**uni_decimals} UNI")

print(f"Your WETH balance: {weth.balanceOf(wallet.address)/10**weth_decimals}")
print(f"Your UNI balance: {uni.balanceOf(wallet.address)/10**uni_decimals}")

## Print current ETH balance in Sepolia

In [ ]:
wallet.balance

In [ ]:
wallet.balance

In [ ]:
# https://docs.uniswap.org/contracts/v2/reference/smart-contracts/router-02

# function swapExactETHForTokens(uint amountOutMin, address[] calldata path, address to, uint deadline)
#   external
#   payable
#   returns (uint[] memory amounts);



# deadline = chain.pending_timestamp + 3600
# eth_amount = 50000000000000000
# swap_path = [weth, wusdc]
# # gas=40000

# # receipt = swap_router.swapExactETHForTokens('amountOutMinimum': 0, 'path':swap_path, 'to':wallet, 'value':eth_amount,  'deadline':deadline)

# receipt = swap_router.swapExactETHForTokens( 0,  [weth, wusdc], wallet, deadline )

# receipt


# Did not work ---- 
# ApeAttributeError: 'SwapRouter02' has no attribute 'swapExactETHForTokens'.

  

#### https://docs.apeworx.io/ape/stable/userguides/contracts.html
receipt = contract.withdraw(sender=account, value=123)
print(receipt.gas_used)

NOTE: You can also use "smart" values such as `"0.1 ether"` or `"100 gwei"`:
_ = contract.withdraw(sender=account, value="0.1 ether")

In [ ]:
# receipt = swap_router(sender=wallet, value="0.01 ether", gas=40000)
# receipt

In [ ]:
# https://sepolia.etherscan.io/address/0x3bFA4769FB09eefC5a80d6E87c3B9C650f7Ae48E#writeContract
# did not work too
# receipt = swap_router.wrapETH(eth_amount, sender=wallet, gas=40000)
# receipt

In [ ]:
### - https://ethereum.stackexchange.com/questions/111799/programmatically-wrap-eth-by-interacting-with-wrapped-ether-contract-in-web3-py
tx = swap_router.deposit().buildTransaction({
        'chainId': 1,
        'gas': 100000,
        # 'gasPrice': w3.eth.gas_price,
        'gasPrice': 2000000, 
        'nonce': w3.eth.get_transaction_count(use_this_address),
        'value': weth_mint_amount
    })



In [ ]:
    # with accounts.use_sender("dev"):
    #     print(f"Minting {weth_mint_amount/10**weth_decimals} WETH")
    #     weth.deposit(value=weth_mint_amount)
    #     print("Approving WETH for swap")
    #     weth.approve(swap_router.address, weth_mint_amount)
    #     swap_params = {
    #         "tokenIn": weth.address,
    #         "tokenOut": uni.address,
    #         "fee": pool_fee,
    #         "recipient": wallet.address,
    #         "amountIn": weth_mint_amount,
    #         "amountOutMinimum": 0,
    #         "sqrtPriceLimitX96": 0,
    #     }
    #     swap_params = tuple(swap_params.values())
    #     print("Swapping WETH for UNI")
    #     amountOut = swap_router.exactInputSingle(swap_params)
    #     print(f"Successfully minted {uni_mint_amount/10**uni_decimals} UNI")

In [ ]:
swap_params = {
    "tokenIn": weth.address,
    "tokenOut": uni.address,
    "fee": pool_fee,
    "recipient": wallet.address,
    "amountIn": weth_mint_amount,
    "amountOutMinimum": 0,
    "sqrtPriceLimitX96": 0,
}
swap_params = tuple(swap_params.values())
swap_params

In [ ]:
        swap_params = {
            "chainId": chain_id,
            "gas": 2000000,
            "tokenOut": uni.address,
            "fee": pool_fee,
            "recipient": wallet.address,
            "amountIn": weth_mint_amount,
            "amountOutMinimum": 0,
            "sqrtPriceLimitX96": 0,
        }

In [ ]:
# # wrap eth
# tx = weth_contract.functions.deposit().build_transaction({
#         # 'chainId': web3.eth.chain_id,
#         'gas': 2000000,
#         "maxPriorityFeePerGas": web3.eth.max_priority_fee,
#         "maxFeePerGas": 100 * 10**9,
#         'nonce': web3.eth.get_transaction_count(account.address),
#         'value': amount_in, # wrap 1 eth
# })

# signed_transaction = web3.eth.account.sign_transaction(tx, private_key)
# tx_hash = web3.eth.send_raw_transaction(signed_transaction.rawTransaction)

# tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
# print(f"tx hash: {Web3.to_hex(tx_hash)}")
# total_gas_used_buy += tx_receipt["gasUsed"]

# weth_balance = weth_contract.functions.balanceOf(account.address).call()
# print(f"weth balance: {weth_balance / 10**18}")

In [ ]:


# Define the swap parameters
# eth_amount = to_wei(0.05, "ether")  # Amount of ETH to swap
eth_amount = 50000000000000000
usdc_address = ADDRESSES["USDC"][chain.chain_id]
# deadline = chain.time() + 600  # 10 minutes from the current block timestamp
deadline = chain.pending_timestamp + 3600
amountOutMin = 0  # Set to 0 for simplicity, but you should set a minimum amount to avoid front-running

# Path for ETH to USDC swap
# swap_path = [AddressType(w3.eth.default_account), usdc_address]
swap_path = [usdc_address]

# Perform the swap
with accounts.use_sender(wallet):
    print(f"Swapping {eth_amount / 10**18} ETH for USDC")
    tx = swap_router.swapExactETHForTokens(
        amountOutMin,
        swap_path,
        wallet.address,
        deadline,
        value=eth_amount
    )
    tx.wait_for_receipt()  # Wait for the transaction to be mined
    print("Swap transaction completed")

# Check the USDC balance of the wallet
usdc = Contract(ADDRESSES["USDC"][chain.chain_id])
usdc_balance = usdc.balanceOf(wallet.address)
print(f"Your USDC balance: {usdc_balance / 10**usdc.decimals()} USDC")

# Check the ETH balance of the wallet
eth_balance = wallet.balance
eth_balance_in_ether = eth_balance / 10**18
print(f"Your ETH balance: {eth_balance_in_ether} ETH")


In [16]:
with accounts.use_sender("giza1"):
    print(f"Minting {weth_mint_amount/10**weth_decimals} WETH")
    weth.deposit(value=weth_mint_amount)
    print("Approving WETH for swap")
    weth.approve(swap_router.address, weth_mint_amount)
    swap_params = {
        "tokenIn": weth.address,
        "tokenOut": wusdc.address,
        "fee": pool_fee,
        "recipient": wallet.address,
        "amountIn": weth_mint_amount/2,
        "amountOutMinimum": 0,
        "sqrtPriceLimitX96": 0,
    }
    swap_params = tuple(swap_params.values())
    print("Swapping WETH for USDC")
    amountOut = swap_router.exactInputSingle(swap_params)
    print(f"Successfully minted {uni_mint_amount/10**uni_decimals} USDC")

print(f"Your WETH balance: {weth.balanceOf(wallet.address)/10**weth_decimals}")
print(f"Your WUSDC balance: {wusdc.balanceOf(wallet.address)/10**wusdc_decimals}")

Minting 0.01 WETH
INFO: Submitted https://sepolia.etherscan.io/tx/0xa6fae58a12e56163f9a2b2f01eb302f73570784e0ee2ea64f6a59902f5f50874


Confirmations (2/2): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:28<00:00, 14.35s/it]


ProviderError: 400 Client Error: Bad Request for url: https://eth-sepolia.g.alchemy.com/v2/alcht_qTqyinch9eBi0vF09MR064gABbQbLI

In [ ]:
# try USDC deposit now.

with accounts.use_sender("giza1"):
    print(f"Minting {weth_mint_amount/10**wusdc_decimals} WUSDC")
    wusdc.deposit(value=weth_mint_amount/2)

In [17]:
with accounts.use_sender("giza1"):
    # print(f"Minting {weth_mint_amount/10**weth_decimals} WETH")
    # weth.deposit(value=weth_mint_amount)
    # print("Approving WETH for swap")
    weth.approve(swap_router.address, weth_mint_amount)
    swap_params = {
        "tokenIn": weth.address,
        "tokenOut": wusdc.address,
        "fee": pool_fee,
        "recipient": wallet.address,
        "amountIn": weth_mint_amount/2,
        "amountOutMinimum": 0,
        "sqrtPriceLimitX96": 0,
    }
    swap_params = tuple(swap_params.values())
    print("Swapping WETH for USDC")
    amountOut = swap_router.exactInputSingle(swap_params)
    print(f"Successfully minted {uni_mint_amount/10**uni_decimals} USDC")

print(f"Your WETH balance: {weth.balanceOf(wallet.address)/10**weth_decimals}")
print(f"Your WUSDC balance: {wusdc.balanceOf(wallet.address)/10**wusdc_decimals}")

INFO: Submitted https://sepolia.etherscan.io/tx/0x1fd24a91db42f1d8ee66f44cd26fd145243d115c326eacdc9221bc7ab75be31f


Confirmations (2/2): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:21<00:00, 10.77s/it]


ProviderError: 400 Client Error: Bad Request for url: https://eth-sepolia.g.alchemy.com/v2/alcht_qTqyinch9eBi0vF09MR064gABbQbLI

In [19]:
pool_fee

3000

In [20]:
with accounts.use_sender("giza1"):
    # print(f"Minting {weth_mint_amount/10**weth_decimals} WETH")
    # weth.deposit(value=weth_mint_amount)
    # print("Approving WETH for swap")
    # weth.approve(swap_router.address, weth_mint_amount)
    swap_params = {
        "tokenIn": weth.address,
        "tokenOut": wusdc.address,
        "fee": pool_fee,
        "recipient": wallet.address,
        "amountIn": weth_mint_amount,
        "amountOutMinimum": 0,
        "sqrtPriceLimitX96": 0,
    }
    swap_params = tuple(swap_params.values())
    print("Swapping WETH for USDC")
    amountOut = swap_router.exactInputSingle(swap_params)
    print(f"Successfully minted {uni_mint_amount/10**uni_decimals} USDC")

print(f"Your WETH balance: {weth.balanceOf(wallet.address)/10**weth_decimals}")
print(f"Your WUSDC balance: {wusdc.balanceOf(wallet.address)/10**wusdc_decimals}")

Swapping WETH for USDC
INFO: Submitted https://sepolia.etherscan.io/tx/0xd48ce69b540e608adb89d75466d434e843fb93b2276c1cbcd6ff2c4b89839caf


Confirmations (2/2): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:29<00:00, 14.54s/it]


ProviderError: 400 Client Error: Bad Request for url: https://eth-sepolia.g.alchemy.com/v2/alcht_qTqyinch9eBi0vF09MR064gABbQbLI